<a href="https://colab.research.google.com/github/arpitpandey2/basicneuralnetwork/blob/main/basicneuralnetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [222]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [223]:
class Model(nn.Module):
  def __init__(self,in_features=4,h1=8,h2=9,out_features=3):
    super().__init__()
    self.fc1= nn.Linear(in_features,h1)
    self.fc2= nn.Linear(h1,h2)
    self.out= nn.Linear(h2,out_features)

  def forward(self,x):
    x= f.relu(self.fc1(x))
    x= f.relu(self.fc2(x))
    x= self.out(x)

    return x

In [224]:
torch.manual_seed(30)
mode1= Model()


In [ ]:
url= 'https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv'
my_df= pd.read_csv(url)
my_df

In [ ]:
my_df['species']=my_df['species'].replace('setosa',0.0)
my_df['species']=my_df['species'].replace('versicolor',1.0)
my_df['species']=my_df['species'].replace('virginica',2.0)
my_df['species'] = my_df['species'].astype(int)
my_df

In [227]:
X=my_df.drop('species',axis=1)
Y=my_df['species']
x=X.values
y=Y.values

In [ ]:
x
y

In [229]:
from sklearn.model_selection import train_test_split


In [230]:
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2, random_state=40)

In [ ]:
y_train.dtype

In [232]:
x_train=torch.FloatTensor(x_train)
x_test=torch.FloatTensor(x_test)

y_train=torch.LongTensor(y_train)
y_test=torch.LongTensor(y_test)

In [233]:
criterion= nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(mode1.parameters(), lr=0.01)

In [ ]:
epochs=100
losses=[]
for i in range(epochs):
  y_pred= mode1.forward(x_train)
  loss= criterion(y_pred,y_train)
  losses.append(loss.detach().numpy())

  if i%10==0:
    print(f"Epoch: {i} and loss {loss}")

  # Backprop and optimize
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

In [ ]:
plt.plot(range(epochs),losses)
plt.ylabel('Loss')
plt.xlabel('Epoch')